In [9]:
pip install dash plotly

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install jupyter-dash

Note: you may need to restart the kernel to use updated packages.


In [11]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px 
import pandas as pd 

df = pd.read_csv("electricity_prices_2025_02_DK2.csv")

df['date'] = pd.to_datetime(df['date'])

df['datetime'] = df['date'].astype(str) + ' ' + df['time_start'].str.split('T').str[1]
df['datetime'] = pd.to_datetime(df['datetime'])

df['hour'] = df['time_start'].str.split('T').str[1].str.split(':').str[0].astype(int)

app = Dash(__name__)

app.layout = html.Div([
    html.H1("Electricity Prices Dashboard", style={'textAlign': 'center'}),

    html.Label("Select Date Range:"),
    dcc.DatePickerRange(
        id='date-range-picker',
        start_date=df['date'].min(),
        end_date=df['date'].max(),
        display_format='YYYY-MM-DD'
    ),
    
    dcc.Graph(id='price-time-series'),
    
    dcc.Graph(id='price-histogram'),
    
    html.Label("Select Hour Range:"),
    dcc.RangeSlider(
        id='hour-range-slider',
        min=0,
        max=23,
        step=1,
        marks={i: str(i) for i in range(24)},
        value=[0, 23]
    )
])

@app.callback(
    [Output('price-time-series', 'figure'),
     Output('price-histogram', 'figure')],
    [Input('date-range-picker', 'start_date'),
     Input('date-range-picker', 'end_date'),
     Input('hour-range-slider', 'value')]
)
def update_graphs(start_date, end_date, hour_range):
    filtered_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    filtered_df = filtered_df[(filtered_df['hour'] >= hour_range[0]) & (filtered_df['hour'] <= hour_range[1])]

    time_series_fig = px.line(
        filtered_df,
        x='datetime',
        y='EUR_per_kWh',
        title='Electricity Prices Over Time',
        labels={'EUR_per_kWh': 'Price (EUR/kWh)', 'datetime': 'Date and Time'}
    )
    
    histogram_fig = px.histogram(
        filtered_df,
        x='EUR_per_kWh',
        title='Distribution of Electricity Prices',
        labels={'EUR_per_kWh': 'Price (EUR/kWh)'}
    )
    
    return time_series_fig, histogram_fig

if __name__ == '__main__':
    app.run(debug=True)